In [1]:
import os
import sys
import numpy as np
import pandas as pd

from datetime import datetime

import lightgbm as lgb

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

import optuna

import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
from src.paths import TRANSFORMED_DATA_DIR, RESIDUALS_DATA_DIR
from src.evaluation import evaluate_metrics, save_metrics

In [4]:
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29,1,0.0
1,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2023-01-30,1,0.0
2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2023-01-31,1,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-02-01,1,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,1.0,0.0,2.0,0.0,0.0,3.0,3.0,0.0,3.0,2.0,...,2.0,2.0,1.0,2.0,3.0,1.0,3.0,2023-12-27,265,3.0
89301,5.0,7.0,2.0,1.0,0.0,1.0,2.0,2.0,3.0,3.0,...,6.0,4.0,2.0,4.0,10.0,3.0,3.0,2023-12-28,265,1.0
89302,5.0,3.0,2.0,3.0,1.0,3.0,1.0,5.0,3.0,1.0,...,3.0,1.0,8.0,5.0,1.0,0.0,6.0,2023-12-29,265,5.0
89303,3.0,4.0,9.0,4.0,1.0,2.0,0.0,0.0,0.0,2.0,...,6.0,3.0,2.0,2.0,5.0,1.0,5.0,2023-12-30,265,2.0


In [5]:
from src.data_split import train_test_split

In [6]:
X_train, y_train, X_test, y_test = train_test_split(df, 
                                                    cutoff_date=datetime(2023, 6, 1, 0, 0, 0),
                                                    target_column_name='target_rides_next_hour')

In [7]:
print(f'Datasets shapes')
print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

Datasets shapes
X_train: (32595, 674)
y_train: (32595,)
X_test: (56710, 674)
y_test: (56710,)


In [8]:
from src.model import get_pipeline

In [9]:
def objective(trial: optuna.trial.Trial) -> float:

    '''
    Define the objective function to optimize the hyperparameters of the model
    Split data into folds for training and validation
    '''

    # Define the hyperparameters to optimize
    hyperparameters = {
        'metric': 'mae',
        'verbose': -1,
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 100),
    }

    tss = KFold(n_splits=3)
    scores = []

    for train_index, val_index in tss.split(X_train):
        # Split the data into training and validation sets
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        # Train the model
        pipeline = get_pipeline(**hyperparameters)
        pipeline.fit(X_train_, y_train_)

        # Evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)
        scores.append(mae)

    return np.array(scores).mean()

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-05-20 16:37:22,716] A new study created in memory with name: no-name-eed932b3-b1f8-45f8-8c29-99efca951263


[LightGBM] [Warning] bagging_fraction is set=0.392995726851379, subsample=1.0 will be ignored. Current value: bagging_fraction=0.392995726851379
[LightGBM] [Warning] feature_fraction is set=0.47876925473391774, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47876925473391774
[LightGBM] [Warning] bagging_fraction is set=0.392995726851379, subsample=1.0 will be ignored. Current value: bagging_fraction=0.392995726851379
[LightGBM] [Warning] feature_fraction is set=0.47876925473391774, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47876925473391774
[LightGBM] [Warning] bagging_fraction is set=0.392995726851379, subsample=1.0 will be ignored. Current value: bagging_fraction=0.392995726851379
[LightGBM] [Warning] feature_fraction is set=0.47876925473391774, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47876925473391774


[I 2024-05-20 16:37:41,548] Trial 0 finished with value: 2.577514515351623 and parameters: {'num_leaves': 43, 'feature_fraction': 0.47876925473391774, 'bagging_fraction': 0.392995726851379, 'min_child_samples': 25}. Best is trial 0 with value: 2.577514515351623.


[LightGBM] [Warning] bagging_fraction is set=0.6112575881703828, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6112575881703828
[LightGBM] [Warning] feature_fraction is set=0.906609712394846, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.906609712394846
[LightGBM] [Warning] bagging_fraction is set=0.6112575881703828, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6112575881703828
[LightGBM] [Warning] feature_fraction is set=0.906609712394846, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.906609712394846
[LightGBM] [Warning] bagging_fraction is set=0.6112575881703828, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6112575881703828
[LightGBM] [Warning] feature_fraction is set=0.906609712394846, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.906609712394846


[I 2024-05-20 16:38:01,098] Trial 1 finished with value: 2.581663449350201 and parameters: {'num_leaves': 40, 'feature_fraction': 0.906609712394846, 'bagging_fraction': 0.6112575881703828, 'min_child_samples': 14}. Best is trial 0 with value: 2.577514515351623.


...

[I 2024-05-20 17:38:07,794] Trial 97 finished with value: 2.5905058310599047 and parameters: {'num_leaves': 243, 'feature_fraction': 0.5216668775884059, 'bagging_fraction': 0.817021977659565, 'min_child_samples': 65}. Best is trial 52 with value: 2.52150880697246.


[LightGBM] [Warning] bagging_fraction is set=0.24522179769008418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24522179769008418
[LightGBM] [Warning] feature_fraction is set=0.9500912543125988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9500912543125988
[LightGBM] [Warning] bagging_fraction is set=0.24522179769008418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24522179769008418
[LightGBM] [Warning] feature_fraction is set=0.9500912543125988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9500912543125988
[LightGBM] [Warning] bagging_fraction is set=0.24522179769008418, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24522179769008418
[LightGBM] [Warning] feature_fraction is set=0.9500912543125988, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9500912543125988


[I 2024-05-20 17:38:33,257] Trial 98 finished with value: 2.635399997978187 and parameters: {'num_leaves': 84, 'feature_fraction': 0.9500912543125988, 'bagging_fraction': 0.24522179769008418, 'min_child_samples': 75}. Best is trial 52 with value: 2.52150880697246.


[LightGBM] [Warning] bagging_fraction is set=0.995617931276432, subsample=1.0 will be ignored. Current value: bagging_fraction=0.995617931276432
[LightGBM] [Warning] feature_fraction is set=0.8849754932767284, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8849754932767284
[LightGBM] [Warning] bagging_fraction is set=0.995617931276432, subsample=1.0 will be ignored. Current value: bagging_fraction=0.995617931276432
[LightGBM] [Warning] feature_fraction is set=0.8849754932767284, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8849754932767284
[LightGBM] [Warning] bagging_fraction is set=0.995617931276432, subsample=1.0 will be ignored. Current value: bagging_fraction=0.995617931276432
[LightGBM] [Warning] feature_fraction is set=0.8849754932767284, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8849754932767284


[I 2024-05-20 17:38:59,640] Trial 99 finished with value: 2.647350747104285 and parameters: {'num_leaves': 88, 'feature_fraction': 0.8849754932767284, 'bagging_fraction': 0.995617931276432, 'min_child_samples': 97}. Best is trial 52 with value: 2.52150880697246.


In [11]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 241, 'feature_fraction': 0.9173905717971508, 'bagging_fraction': 0.8850277325059562, 'min_child_samples': 70}


In [12]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.8850277325059562, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8850277325059562
[LightGBM] [Warning] feature_fraction is set=0.9173905717971508, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9173905717971508


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x13584f820>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.8850277325059562,
                               feature_fraction=0.9173905717971508,
                               min_child_samples=70, num_leaves=241))])

In [13]:
predictions = pipeline.predict(X_test)

In [14]:
residuals = y_test - predictions

residuals.to_csv(RESIDUALS_DATA_DIR / 'residuals_lightgmb_3.csv', index=False)

In [15]:
metrics = evaluate_metrics(y_test, predictions)

for metric, value in metrics.items():
    print(f'{metric}: {value:.2f}')

MAE: 2.91
MSE: 127.62
RMSE: 11.30
SMAPE: 149.86
R: 0.96
R-squared: 0.92


In [16]:
model_name = 'LightGBM with Hyperparameter Tuning'
metrics['Model'] = model_name
save_metrics(metrics)

In [17]:
%reload_ext autoreload
%autoreload 2

In [18]:
from src.plot import plot_one_sample

In [19]:
plot_one_sample(
    example_id=16849,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [20]:
plot_one_sample(
    example_id=56700,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [21]:
max_value = predictions.max()
argmax = predictions.argmax()
print(f'Max value of the predictions: {max_value:.2f}')
print(f'Index of the max value: {argmax}')

Max value of the predictions: 522.14
Index of the max value: 16701
